In [1]:
import bidict
import os
import csv
import scipy.sparse
import numpy as np

In [2]:
def get_chembls(filename):
    result = []
    first_line = True
    with open(filename, 'r') as f:
        for line in f:
            if first_line:
                assert 'CHEMBL' in line
                result.append(line.strip())
                first_line = False
            elif '$$$$' in line:
                first_line=True
    return result

In [3]:
def get_mapping(all_compounds_file):
    mapping = bidict.bidict()
    all_chembls = get_chembls(all_compounds_file)        
    mapping.update(dict(zip(xrange(len(all_chembls)), all_chembls)))

    return mapping

In [4]:
def get_all_files(path):
    return [os.path.join(path,filename) for filename in os.listdir(path) if os.path.isfile(os.path.join(path, filename))]

In [25]:
def load_similarity_matrices():
    bin_similarity = []
    scale_similarity = []
    row_ind = []
    col_ind = []
    
    mapping_idx_chembl = get_mapping('Random_compounds_100.sdf')
    n_compunds = len(mapping_idx_chembl)
    dict_idx_chemblchembl = \
    dict( [ [int(''.join(c for c in filename if c.isdigit() )), tuple(get_chembls(filename))] \
          for filename in get_all_files('pairs') ] )
    
    print len(dict_idx_chemblchembl), 'liczba par'
    
    # czy pary są unikalne?
    ile = 0
    non_unique = {}
    print 'chembl, chembl, numer pary, numer innej pary'
    for key in sorted(dict_idx_chemblchembl.keys()):
        i, j = dict_idx_chemblchembl[key]
        for another_key in sorted(dict_idx_chemblchembl.keys()):
            if i in dict_idx_chemblchembl[another_key] and j in dict_idx_chemblchembl[another_key]:
                if key != another_key:
                    print i, j, key, another_key
                    dict_key = min(key, another_key)
                    dict_value =[max(key, another_key)]
                    if dict_value == None:
                        print 'tutaj tuaja', key, another_key
                    if dict_key not in non_unique.keys():
                        non_unique[dict_key] = dict_value
                    else:
                        if non_unique[dict_key] is None:
                            print non_unique
                            print dict_key
                            break
                        non_unique[dict_key] = non_unique[dict_key].extend(dict_value)
                    ile +=1
    print 'liczba alertów', ile
    print non_unique
    print non_unique.values()
    
    with open('Similarity.csv', 'r') as csvfile:
        for pair_number, bin_sim, scale_sim in csv.reader(csvfile, delimiter=','):
            chembl_i, chembl_j = dict_idx_chemblchembl[int(pair_number)]
            chembl_i_idx, chembl_j_idx = mapping_idx_chembl.inv[chembl_i], mapping_idx_chembl.inv[chembl_j]
            row_ind.extend([chembl_i_idx, chembl_j_idx])
            col_ind.extend([chembl_j_idx, chembl_i_idx])
            bin_similarity.extend([int(bin_sim), int(bin_sim)])
            scale_similarity.extend([int(scale_sim), int(scale_sim)])
    
    assert len(dict_idx_chemblchembl)*2 == len(bin_similarity)
    assert len(bin_similarity)==len(scale_similarity)
    assert len(scale_similarity)==len(row_ind)
    assert len(row_ind)==len(col_ind)
    
    print len(row_ind)
     
    # we want bin similarities to be -1, 1 not 0, 1
    bin_similarity = [(-1)**(1-x) for x in bin_similarity]
    assert 0 not in scale_similarity
    assert 0 not in bin_similarity
    
    scale_similarity = scipy.sparse.csr_matrix((scale_similarity, (row_ind, col_ind)), shape=(n_compunds, n_compunds))
    bin_similarity   = scipy.sparse.csr_matrix((bin_similarity,   (row_ind, col_ind)), shape=(n_compunds, n_compunds))
    
    assert scale_similarity.nnz == len(row_ind)
    assert bin_similarity.nnz == len(row_ind) 
    
    assert np.all(scale_similarity.todense() == np.transpose(scale_similarity.todense()))
    assert scale_similarity.shape == (n_compunds, n_compunds)
    assert np.all(bin_similarity.todense() == np.transpose(bin_similarity.todense()))
    assert bin_similarity.shape == (n_compunds, n_compunds)
    
    assert scale_similarity.nnz == bin_similarity.nnz
    assert np.all(scale_similarity.indices == bin_similarity.indices)
    #print scale_similarity.nonzero() == bin_similarity.nonzero()
    
    return bin_similarity, scale_similarity, mapping_idx_chembl

SyntaxError: EOL while scanning string literal (<ipython-input-25-3d611574466d>, line 26)

In [24]:
bin_sim, scale_sim, mapping_idx_chembl = load_similarity_matrices()

450 liczba par
chembl, chembl, numer pary, numer innej pary
CHEMBL426317 CHEMBL2397911 11 267
CHEMBL12314 CHEMBL14563 16 201
CHEMBL1255837 CHEMBL76307 25 29
CHEMBL1255837 CHEMBL76307 29 25
CHEMBL289436 CHEMBL364534 38 242
CHEMBL41389 CHEMBL41792 60 417
CHEMBL14841 CHEMBL472789 94 416
CHEMBL289436 CHEMBL598443 105 257
CHEMBL88894 CHEMBL475778 124 224
CHEMBL2397911 CHEMBL14374 165 441
CHEMBL2087337 CHEMBL598443 167 316
CHEMBL2087337 CHEMBL598443 167 411
CHEMBL472986 CHEMBL220900 195 391
CHEMBL201930 CHEMBL55914 198 314
CHEMBL12314 CHEMBL14563 201 16
CHEMBL223468 CHEMBL2397893 212 293
CHEMBL14076 CHEMBL209324 215 269
CHEMBL88894 CHEMBL475778 224 124
CHEMBL364534 CHEMBL289436 242 38
CHEMBL209324 CHEMBL19215 252 271
CHEMBL289436 CHEMBL598443 257 105
CHEMBL426317 CHEMBL2397911 267 11
CHEMBL14076 CHEMBL209324 269 215
CHEMBL19215 CHEMBL209324 271 252
CHEMBL12314 CHEMBL1255837 274 318
CHEMBL215468 CHEMBL263881 289 320
CHEMBL2397893 CHEMBL223468 293 212
CHEMBL220900 CHEMBL41792 301 390
CHEMBL201

AssertionError: 

In [ ]:
print bin_sim.nonzero() == scale_sim.nonzero()

In [ ]:
print bin_sim.nnz
print scale_sim.nnz

a, b = bin_sim.nonzero()[0], bin_sim.nonzero()[1]
c, d = scale_sim.nonzero()[0], scale_sim.nonzero()[1]

print a.shape, c.shape
print b.shape, d.shape

In [ ]:
print len(get_mapping('pairs'))

In [ ]:
print get_chembls('pairs/Random_pair11.sdf')
print get_chembls('pairs/Random_pair267.sdf')